# <center>Car Maker Classification (MobileNet Keras)</center>

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
keras.__version__

'2.7.0'

## Run on GPU

In [3]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# print("Num GPUs Available: ", len(physical_devices))
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Preparing images

In [10]:
train_path = 'dataset/train_spl'
valid_path = 'dataset/val_spl'
test_path = 'dataset/test_spl'

train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=train_path, target_size=(224,224), batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=valid_path, target_size=(224,224), batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(
    directory=test_path, target_size=(224,224), batch_size=10, shuffle=False)

Found 9791 images belonging to 196 classes.
Found 3312 images belonging to 196 classes.
Found 3082 images belonging to 196 classes.


## Choosing Model

In [14]:
model_original = tf.keras.applications.mobilenet.MobileNet()

In [7]:
# model_original.summary() # to show all models layers

In [14]:
# taking all layers except last 6 layers
x = model_original.layers[-6].output
output = Dense(units=196, activation='softmax')(x)

In [15]:
model = Model(inputs=model_original.input, outputs=output)

In [16]:
# freezing last n layers (n is an arbitrary number)
for layer in model.layers[:-23]:
    layer.trainable = False

In [8]:
# model.summary() # to show all models layers

## Training Model

In [18]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# makeing checkpoint each epoch
checkpoint = ModelCheckpoint("M_MobileNet_checkpoints/M_MobileNet_checkpoint.hdf5", monitor='loss', verbose=0,
    save_best_only=False, mode='auto', save_freq=1)

In [19]:
model.fit(x=train_batches,
            steps_per_epoch=len(train_batches),
            validation_data=valid_batches,
            validation_steps=len(valid_batches),
            epochs=30,
            verbose=2,
            callbacks=[checkpoint]
)

Epoch 1/30
980/980 - 260s - loss: 4.2605 - accuracy: 0.1514 - val_loss: 3.0160 - val_accuracy: 0.3493
Epoch 2/30
980/980 - 263s - loss: 2.1725 - accuracy: 0.5848 - val_loss: 1.9574 - val_accuracy: 0.5655
Epoch 3/30
980/980 - 257s - loss: 1.2379 - accuracy: 0.7946 - val_loss: 1.5008 - val_accuracy: 0.6540
Epoch 4/30
980/980 - 253s - loss: 0.7565 - accuracy: 0.8966 - val_loss: 1.2561 - val_accuracy: 0.7020
Epoch 5/30
980/980 - 253s - loss: 0.4570 - accuracy: 0.9543 - val_loss: 1.1081 - val_accuracy: 0.7228
Epoch 6/30
980/980 - 253s - loss: 0.2867 - accuracy: 0.9809 - val_loss: 1.0414 - val_accuracy: 0.7322
Epoch 7/30
980/980 - 253s - loss: 0.1723 - accuracy: 0.9929 - val_loss: 0.9763 - val_accuracy: 0.7470
Epoch 8/30
980/980 - 253s - loss: 0.1201 - accuracy: 0.9945 - val_loss: 0.9667 - val_accuracy: 0.7461
Epoch 9/30
980/980 - 254s - loss: 0.0899 - accuracy: 0.9949 - val_loss: 0.9172 - val_accuracy: 0.7639
Epoch 10/30
980/980 - 256s - loss: 0.0681 - accuracy: 0.9955 - val_loss: 0.9179 - 

## Saving Model

In [28]:
model.save("M_MobileNet.h5")

## Result Visualization

In [20]:
test_labels = test_batches.classes

In [21]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [4]:
# def plot_confusion_matrix(cm, classes,
#                           normalize=False,
#                           title='Confusion matrix',
#                           cmap=plt.cm.Blues):
#     """
#     This function prints and plots the confusion matrix.
#     Normalization can be applied by setting `normalize=True`.
#     """
#     plt.rcParams["figure.figsize"] = (12,12)
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#             horizontalalignment="center",
#             color="white" if cm[i, j] > thresh else "black")

In [3]:
# cm = confusion_matrix(y_true=test_labels, y_pred=predictions.argmax(axis=1))

In [2]:
# test_batches.class_indices

In [1]:
# cm_plot_labels = list(range(0, 196))
# plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')